In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from pathlib import Path


In [2]:
stock_csv_path = Path('monthly_data.csv')
monthly_df = pd.read_csv(stock_csv_path, index_col = 'Date')
monthly_df_daily_returns = monthly_df.pct_change()

In [4]:
monthly_df_daily_returns.shape

(59, 100)

In [5]:
monthly_df_daily_returns.dropna(inplace = True)
monthly_df_daily_returns.shape

(58, 100)

In [6]:
monthly_df_daily_returns.index = pd.DatetimeIndex(monthly_df_daily_returns.index)

In [8]:
ffm_csv_path = Path('F-F_Research_Data_Factors.csv')
ffm_data = pd.read_csv(ffm_csv_path, index_col='Date')
ffm_data.shape

(58, 4)

In [9]:
ffm_data = ffm_data/100

In [10]:
ffm_data.index = pd.DatetimeIndex(ffm_data.index)

In [11]:
reg = linear_model.LinearRegression()

In [12]:
reg = linear_model.LinearRegression()
fit = reg.fit(X=ffm_data[['SMB','HML','RF']],
              y=monthly_df_daily_returns.AAPL)

In [13]:
fit.coef_

array([ 0.49315924, -0.13456823, 15.50727616])

In [14]:
import statsmodels.api as sm
from statsmodels.regression import linear_model as lm

In [16]:
results_dict = {}
params = {}

for ticker in monthly_df_daily_returns.columns:
    results_dict[ticker] = lm.OLS(endog=monthly_df_daily_returns.loc[:,ticker],
                  exog=sm.add_constant(ffm_data[['Mkt-RF','SMB','HML']])).fit()
    
    params[ticker] = results_dict[ticker].params

C:\Users\nikki\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [18]:
mkt_rf_mean = ffm_data['Mkt-RF'].mean()
smb_mean = ffm_data['SMB'].mean()
hml_mean = ffm_data['HML'].mean()
rf_mean = ffm_data['RF'].mean()

In [19]:
mkt_rf_mean

0.008677586206896553

In [20]:
smb_mean

-0.0014310344827586205

In [21]:
hml_mean

-0.002867241379310345

In [22]:
rf_mean

0.0008482758620689655

In [23]:
ffm_constants = pd.DataFrame.from_dict(params, orient='index',
                       columns=['Mkt-RF','const', 'SMB', 'HML'])
ffm_constants

,Mkt-RF,const,SMB,HML
DXC,1.384191,0.002706,0.217156,-0.809495
BBY,1.079844,0.012832,-0.249594,0.489003
AMG,1.369161,-0.022845,0.188433,0.412117
CNP,0.502963,0.004366,-0.310998,-0.035387
WMT,0.706901,0.002256,-0.932470,0.052918
...,...,...,...,...
MCO,1.046873,0.009035,0.134072,-0.277574
HII,0.834222,0.006134,-0.167012,0.035447
RL,0.871189,-0.002597,0.564390,0.400176
LNT,0.340003,0.009248,-0.355165,-0.104039


In [24]:
ffm_returns_monthly = ffm_constants['const'] + ffm_constants['Mkt-RF'] * mkt_rf_mean + ffm_constants['SMB'] * smb_mean + ffm_constants['HML'] * hml_mean + rf_mean

In [25]:
ffm_returns_annual = ffm_returns_monthly * 12
ffm_returns_annual

DXC    0.210906
BBY    0.264073
AMG   -0.138798
CNP    0.121502
WMT    0.125055
         ...   
MCO    0.234858
HII    0.172310
RL     0.046267
LNT    0.166244
AXP    0.132252
Length: 100, dtype: float64

In [26]:
ffm_returns_annual.sort_values(ascending = False, inplace = True)
ffm_returns_annual.head(20)

ADBE    0.329761
MSFT    0.322501
CTAS    0.285973
BBY     0.264073
SNPS    0.258979
FISV    0.249903
UNH     0.243094
FB      0.238789
RMD     0.236317
AAPL    0.236049
MCO     0.234858
BSX     0.233861
BA      0.233236
MTD     0.232895
MSI     0.231468
OKE     0.230148
NEE     0.227986
ACN     0.225818
TXN     0.224912
TMO     0.223396
dtype: float64

In [28]:
ffm_returns_annual.to_csv('100ffmreturns.csv')

C:\Users\nikki\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
